In [1]:
import socket
import ssl
import os
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import utils
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


In [2]:

def generate_rsa_keypair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    return private_key

def generate_dh_keypair():
    parameters = dh.generate_parameters(generator=2, key_size=2048)
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key

def derive_dh_shared_key(private_key, public_key):
    shared_key = private_key.exchange(public_key)
    return shared_key

def rsa_encrypt(public_key, plaintext):
    ciphertext = public_key.encrypt(
        plaintext.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext

def rsa_decrypt(private_key, ciphertext):
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return plaintext.decode()

def aes_encrypt(key, plaintext):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()
    return iv + ciphertext

def aes_decrypt(key, ciphertext):
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext.decode()

def start_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 8443))
    server_socket.listen(1)

    print("Server listening on port 8443...")

    while True:
        client_socket, client_address = server_socket.accept()
        print(f"Connection from {client_address} has been established!")

        context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
        print("SSL/TLS context created.")
        context.load_cert_chain(certfile="server.crt", keyfile="server.key")
        print("SSL/TLS context loaded.")
        
        with context.wrap_socket(client_socket, server_side=True) as ssock:
            print("SSL/TLS connection established.")
            encryption_choice = ssock.recv(1024).decode()
            print(f"Client chose encryption method {encryption_choice}")
            
            
            
            # res = input("Enter response to client: ")

            # ssock.send(res.encode())

            # encryption_choice = ssock.recv(1024).decode()

            if encryption_choice == "1":
                # RSA encryption and PKI key exchange
                pb_key = ssock.recv(1024)
                print(f"Key received from client: {pb_key.decode()}") 
                
                private_key = generate_rsa_keypair()
                public_key = private_key.public_key()
                
                #send public key to client
                ssock.send(public_key.public_bytes(
                    encoding=serialization.Encoding.PEM,
                    format=serialization.PublicFormat.SubjectPublicKeyInfo
                ))
                
                
                client_public_key = load_pem_public_key(pb_key, backend=default_backend())
                print(f"Client public key: {client_public_key}")
                
                # shared_secret = private_key.exchange(client_public_key)
                # message = "This is a secure RSA message."
                
                rec_msg = ssock.recv(1024)
                print(f"Message received from client: {rec_msg}")
                decrypted_msg = rsa_decrypt(private_key, rec_msg)
                print(f"Decrypted message: {decrypted_msg}")
                return
                
                ciphertext = rsa_encrypt(client_public_key, message)
                ssock.send(ciphertext)

            elif encryption_choice == "2":
                # AES encryption and Diffie-Hellman key exchange
                alice_private_key, alice_public_key = generate_dh_keypair()
                ssock.send(alice_public_key.public_bytes(
                    encoding=serialization.Encoding.PEM,
                    format=serialization.PublicFormat.SubjectPublicKeyInfo
                ))

                received_public_key = ssock.recv(2048)
                received_public_key = serialization.load_pem_public_key(received_public_key, backend=default_backend())
                shared_key = derive_dh_shared_key(alice_private_key, received_public_key)

                message = "This is a secure AES message."
                ciphertext = aes_encrypt(shared_key, message)
                ssock.send(ciphertext)



In [3]:
if __name__ == "__main__":
    start_server()


Server listening on port 8443...
Connection from ('127.0.0.1', 65484) has been established!
SSL/TLS context created.
SSL/TLS context loaded.
SSL/TLS connection established.
Client chose encryption method 1
Key received from client: -----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAxXujozXKymu6Yb09gFFa
Lz05Ab/7p9kSzL/PY15FpVJTIk621J9LLmVgQDn34ybYrxGQdwEmEQR2WyvYiabr
jyZwy/pslOcjIzPun911kfdPgy61aJ51Zt9dKvqtSF3XuTxK4xkTpxCqGzF5vXBK
xNxzR8kKoxz9DUQgVydOSCyJqQF1JBr12fpzusqi2PgYVlShunaVlInOcaCGxfBd
Ef+M5XK/GMSOL7bv7yAqB7iIaTa+EKTwvzXYGqevLHpdVUOzvYFmiuQDohgqz+ND
B/41owFt6Yy/DNXK/G9G1viFjivaPJ4GWkSf9rTsQ7KDyHfJ8DQD7FDEhUtnHnzS
DQIDAQAB
-----END PUBLIC KEY-----

Client public key: <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x0000020124455780>
Message received from client: b'\x01\x0c\xc9\x92\x14\xeav7\xf0\x85\x9er\x07\xcf\r`-y\x9f\xc3Y\x02a\x98k\xd3\xd4\x85\x90\xb3\xb2\xc3\xfb\xc12\x891\xa9\x15\xed,\xa9jD\xd5C-\x9dA\xebi\x97\x15D=\xd9\x1a\x1f\xdb\x9c\

In [4]:
# # Generate a private key for the server
# openssl genpkey -algorithm RSA -out server.key

# # Create a Certificate Signing Request (CSR) for the server
# openssl req -new -key server.key -out server.csr

# # Sign the server CSR with the CA certificate
# openssl x509 -req -in server.csr -CA ca.crt -CAkey ca.key -CAcreateserial -out server.crt
